In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from src.helpers.cumsum_mps_fix import reassuring_message
print(reassuring_message) # avoid "unused" import :P

monkey-patched cumsum to fallback to CPU, for compatibility on MPS backend; see https://github.com/pytorch/pytorch/issues/89784


In [11]:
#%env PYTORCH_ENABLE_MPS_FALLBACK=1

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "mps" if torch.backends.mps.is_available() else "cpu"
#device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = "[PAD]"
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(device)


In [19]:
prompt = "Hi, "

ids = tokenizer(prompt, padding=True, return_tensors="pt").to(device)

with torch.no_grad():
    gen_tokens = model.generate(
            ids.input_ids,
            attention_mask=ids.input_ids.ne(tokenizer.pad_token_id).long(),
            pad_token_id=50256,
            max_length=30,
            do_sample=False,
            #temperature=1.0,
            use_cache=True
        )

gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
print(gen_text)

attention_mask tensor([[1, 1, 1]])
cumsum tensor([[1, 2, 3]])
attention_mask tensor([[1, 1, 1, 1]])
cumsum tensor([[1, 2, 3, 4]])
attention_mask tensor([[1, 1, 1, 1, 1]])
cumsum tensor([[1, 2, 3, 4, 5]])
attention_mask tensor([[1, 1, 1, 1, 1, 1]])
cumsum tensor([[1, 2, 3, 4, 5, 6]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1]])
cumsum tensor([[1, 2, 3, 4, 5, 6, 7]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1]])
cumsum tensor([[1, 2, 3, 4, 5, 6, 7, 8]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])
cumsum tensor([[1, 2, 3, 4, 5, 6, 7, 8, 9]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
cumsum tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
cumsum tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
cumsum tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]])
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
cumsum tenso

In [2]:
import torch

example_tensor = torch.tensor([[1]])
print(example_tensor.cumsum(0))
print(example_tensor.cumsum(1))
print(example_tensor.cumsum(-1))
print(example_tensor.cumsum(-2))

/Users/andrestorhaug/Code/playground/gpt-j-6B-smart-contract-audit/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[1]])
tensor([[1]])
tensor([[1]])
tensor([[1]])


In [7]:
import torch

mps_device = torch.device("mps")
z = torch.ones(5,2, device=mps_device)
print(z)
print(z.cumsum(1))
print(z.sort(dim=0))

tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]], device='mps:0')
tensor([[1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.],
        [1., 2.]], device='mps:0')


NotImplementedError: The operator 'aten::sort.values_stable' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [4]:
ids.attention_mask.long().cumsum(-1)

NameError: name 'ids' is not defined

In [13]:
tokenizer(prompt, padding=True, return_tensors="pt")

{'input_ids': tensor([[   27,    91, 22390,    91,    29, 28484, 15291, 30642,   318,   412,
          7397,  1238,  1391]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [36]:
ids.input_ids.ne(tokenizer.pad_token_id).long()

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [34]:
tokenizer.pad_token_id

In [23]:
%%html
<style>
.cell-output-ipywidget-background {
   background-color: transparent !important;
}
.jp-OutputArea-output {
   background-color: transparent;
}  
</style>

In [43]:
import torch
xs = torch.arange(30).to('mps')
print(xs.sum(0))

tensor(435, device='mps:0')


In [57]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

prompt = "Hell! My name is André. What is "
inputs = tokenizer(prompt, return_tensors="pt").to(device)

generation_output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/Users/andrestorhaug/Code/playground/gpt-j-6B-smart-contract-audit/.venv/lib/python3.9/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


RuntimeError: Expected dim to be between 0 and 2 but got -1

In [17]:
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [40]:
generation_output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)

/Users/andrestorhaug/Code/playground/gpt-j-6B-smart-contract-audit/.venv/lib/python3.9/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
